In [76]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score

from perceptron import Perceptron

In [77]:
df = pd.read_csv('Inputs/Surgical-deepnet.csv')
df.head()

,bmi,Age,asa_status,baseline_cancer,baseline_charlson,baseline_cvd,baseline_dementia,baseline_diabetes,baseline_digestive,baseline_osteoart,...,complication_rsi,dow,gender,hour,month,moonphase,mort30,mortality_rsi,race,complication
0,19.31,59.2,1,1,0,0,0,0,0,0,...,-0.57,3,0,7.63,6,1,0,-0.43,1,0
1,18.73,59.1,0,0,0,0,0,0,0,0,...,0.21,0,0,12.93,0,1,0,-0.41,1,0
2,21.85,59.0,0,0,0,0,0,0,0,0,...,0.00,2,0,7.68,5,3,0,0.08,1,0
3,18.49,59.0,1,0,1,0,0,1,1,0,...,-0.65,2,1,7.58,4,3,0,-0.32,1,0
4,19.70,59.0,1,0,0,0,0,0,0,0,...,0.00,0,0,7.88,11,0,0,0.00,1,0


In [78]:
df.isnull().sum()

bmi                    0
Age                    0
asa_status             0
baseline_cancer        0
baseline_charlson      0
baseline_cvd           0
baseline_dementia      0
baseline_diabetes      0
baseline_digestive     0
baseline_osteoart      0
baseline_psych         0
baseline_pulmonary     0
ahrq_ccs               0
ccsComplicationRate    0
ccsMort30Rate          0
complication_rsi       0
dow                    0
gender                 0
hour                   0
month                  0
moonphase              0
mort30                 0
mortality_rsi          0
race                   0
complication           0
dtype: int64

In [79]:
X = df.drop('complication', axis=1).copy().values
Y = df[['complication']].copy().values

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, shuffle=True)
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

((11708, 24), (2927, 24), (11708, 1), (2927, 1))

In [80]:
model = Perceptron(learning_rate=0.001, input_length=X_train.shape[1], type_of_data='classification')
model.fit(X_train,  X_test, Y_train, Y_test, epochs=128)

100%|██████████| 128/128 [01:30<00:00,  1.42it/s]


In [81]:
Y_pred = model.predict(X_test)
Y_pred = np.where(Y_pred > 0.5, 1, 0)
Y_pred = np.max(Y_pred, axis=1)
Y_pred = Y_pred.reshape(-1, 1)
Y_pred.shape

(2927, 1)

In [82]:
def calculate_precision_and_recall(Y_test, Y_pred):
    FN = FP = TN = TP = 0
    for y_test, y_pred in zip(Y_test, Y_pred):
        if y_test == 0:
            if y_pred == 0:
                TN += 1 # False Negative
            elif y_pred == 1:
                FP += 1 # False Positive
        elif y_test == 1:
            if y_pred == 0:
                FN += 1 # True Negative
            elif y_pred == 1:
                TP += 1 # True Positive 
    
    precision = TP / (TP + FP)
    recall = TP / (TP + FN)

    return precision, recall
    

In [84]:
precision = TP / (TP + FP)
recall = TP / (TP + FN)

precision, recall

(0.6322067594433399, 0.42857142857142855)

In [85]:
my_precision, my_recall = calculate_precision_and_recall(Y_test, Y_pred)

sklearn_precision = precision_score(Y_test, Y_pred, average= "binary")
sklearn_recall = recall_score(Y_test, Y_pred, average= "binary")

In [86]:
my_precision, my_recall, sklearn_precision, sklearn_recall

(0.4669887278582931,
 0.4309063893016345,
 0.4669887278582931,
 0.4309063893016345)